In [16]:
import pandas
import math

In [17]:
total_phases = 12
increasing_or_decreasing = type('', (), {'increasing':'<', 'decreasing':'>'})
pricing_patterns = ('波动型', '递减型', '三期型', '四期型');
pricing_patterns_last_week = []

# K 102
# 7 101
last_sunday_price = 101

# K 82 78 75 133 111 143 155
# 7 88 83 80 127 154 209
this_week_prices = [
    88, # 周一 上午 
    83, # 周一 下午
    80, # 周二上午
    127, # 周二下午
    154, # 周三 上午
    0, # 周三 下午
    0, # 周四 上午
    0, # 周四 下午
    0, # 周五 上午
    0, # 周五 下午
    0, # 周六 上午
    0, # 周六 下午
    0 # 周日 买入
]

In [18]:
basic_rate = this_week_prices[0] / last_sunday_price
basic_rate

0.8712871287128713

In [19]:
def print_range(min_price, max_price):
    return '{0}-{1}'.format(str(min_price), str(max_price))

In [20]:
# 判断第一个下跌/上涨阶段是否已结束，并计算天数
# 周一上午大于上周日买入价格的90%，即视为上涨
def calculate_increased_or_decreased_days(increasing_or_decreasing):
    if this_week_prices[0] != 0 and eval(str(this_week_prices[0]) + increasing_or_decreasing + str((last_sunday_price * 0.9))):
        return 0, True
    else:
        days = 1
        phase_end = False
        for i,j in enumerate(range(1,len(this_week_prices))):
            if this_week_prices[i] != 0 and this_week_prices[j] != 0 and this_week_prices[i] > this_week_prices[j]:
                days = days + 1
            if this_week_prices[i] != 0 and this_week_prices[i] < this_week_prices[j]:
                phase_end = True
                break
        return days, phase_end

In [21]:
# 根据基础倍率、递减倍率以及递减天数计算当期最高价格和最低价格
# 向上取整 math.ceil
# 四舍五入 round
# 向下取整 int
def calculate_max_min_price(basic_rate, adjustment = [0, 0], days = 0):
    max_price = math.ceil(last_sunday_price * (basic_rate[-1] - adjustment[0] * days))
    min_price = math.ceil(last_sunday_price * (basic_rate[0] - adjustment[-1] * days))
    return max_price, min_price

In [22]:
# 计算连续下跌n期的价格
def calculate_continuous_phases_prices(start_date, end_date, basic_rate, adjustment_rates = [0, 0], start_adjest_day = 0):
    phase_price = []
    for i in range(start_date, end_date):
        if this_week_prices[i] != 0:
            phase_price.append(this_week_prices[i])
        else:
            max_price, min_price = calculate_max_min_price(basic_rate, adjustment_rates, (i - start_adjest_day))
            phase_price.append(print_range(min_price, max_price))
    return phase_price

In [23]:
'''
波动型处理
分为5个区间：上涨区间1 - 下跌区间1 - 上涨区间2 - 下跌区间2 - 上涨区间3
3个上涨区间共占有 7 个半天：上涨区间1会占据 0 到 6 个半天，上涨区间2会占据 1 到 7 个半天，上涨区间3占据剩余半天，即上涨区间1和3有可能都不存在
2个下跌区间共占有 5 个半天：两个区间分别为 2+3 或 3+2
'''
def process_increasing_phase(start_date, end_date):
    return calculate_continuous_phases_prices(start_date, end_date, [0.9, 1.4])

def process_decreasing_phase(start_date, end_date, current_basic_rate, adjustment_date = 0):
    return calculate_continuous_phases_prices(start_date, end_date, current_basic_rate, [0.04, 0.1], adjustment_date)

def calculate_random():
    result = []
    max_increasing_period_1_phases = 6
    max_decreasing_period_phases = 5
    
    increased_days, increasing_phase_1_end = calculate_increased_or_decreased_days(increasing_or_decreasing.increasing)
    if increasing_phase_1_end:
        increasing_phase_1 = process_increasing_phase(0, increased_days)
        current_basic_rate = this_week_prices[increased_days] / last_sunday_price
        for j in range(2, 4):
            decreasing_phase_1 = process_decreasing_phase(increased_days, increased_days + j, [current_basic_rate], increased_days)
            decreasing_phase_2 = process_decreasing_phase(increased_days + j, increased_days + max_decreasing_period_phases, [current_basic_rate], increased_days + j)
            for k in range(0, total_phases - max_decreasing_period_phases - increased_days):
                increasing_phase_2 = process_increasing_phase(increased_days + j, increased_days + j + k + 1)
                increasing_phase_3 = process_increasing_phase(1, total_phases - max_decreasing_period_phases - increased_days - k)
                result.append(increasing_phase_1 + decreasing_phase_1 + increasing_phase_2 + decreasing_phase_2 + increasing_phase_3)
    else:
        for i in range(increased_days, max_increasing_period_1_phases + 1):
            increasing_phase_1 = process_increasing_phase(0, i)
            for j in range(2, 4):
                decreasing_phase_1 = process_decreasing_phase(i, i + j, i)
                decreasing_phase_2 = process_decreasing_phase(i + j, i + max_decreasing_period_phases, i + j)
                for k in range(0, total_phases - max_decreasing_period_phases - i):
                    increasing_phase_2 = process_increasing_phase(i + j, i + j + k + 1)
                    increasing_phase_3 = process_increasing_phase(1, total_phases - max_decreasing_period_phases - i - k)
                    result.append(increasing_phase_1 + decreasing_phase_1 + increasing_phase_2 + decreasing_phase_2 + increasing_phase_3)
    return result

In [24]:
'''
递减型处理
在基础倍率基础上固定下跌全程12个半天
'''

def calculate_decreasing():
    return [calculate_continuous_decreasing_phases(0, total_phases,[basic_rate], [0.03, 0.05])]

In [25]:
'''
三期型处理
分为4个区间：下跌区间1 - 上涨区间1 - 下跌区间2 - 下跌区间3
时间上分别是 1 到 7 个半天 - 3 个半天 - 2 个半天 - 剩余半天数量
下面计算中，把 上涨区间1 和 下跌区间2 放在一起处理
'''
# 第一个下跌区间
def process_decreasing_phase_1(start_date, end_date):
    return calculate_continuous_phases_prices(start_date, end_date,[basic_rate], [0.03, 0.05])

# 第一个暴涨区间 和 第二个下跌区间
def process_increasing_phase_1_and_decreasing_phase_2(start_date):
    increasing_rates_1 = [0.9, 1.4]
    increasing_rates_2 = [1.4, 2.0]
    increasing_rates_3 = [2.0, 6.0]
    phase_price = []

    # 上涨第一天
    if this_week_prices[start_date] > 0:
        phase_price.append(this_week_prices[start_date])
    else:
        max_price, min_price = calculate_max_min_price(increasing_rates_1)
        phase_price.append(print_range(min_price, max_price))

    # 上涨第二天
    if this_week_prices[start_date + 1] > 0:
        phase_price.append(this_week_prices[start_date + 1])
    else:
        max_price, min_price = calculate_max_min_price(increasing_rates_2)
        phase_price.append(print_range(min_price, max_price))

    # 上涨第三天
    if this_week_prices[start_date + 2] > 0:
        phase_price.append(this_week_prices[start_date + 2])
    else:
        max_price, min_price = calculate_max_min_price(increasing_rates_3)
        phase_price.append(print_range(min_price, max_price))    
    
    # 下跌第一天
    if this_week_prices[start_date + 3] > 0:
        phase_price.append(this_week_prices[start_date + 3])
    else:
        max_price, min_price = calculate_max_min_price(increasing_rates_2)
        phase_price.append(print_range(min_price, max_price))

    # 下跌第二天
    if this_week_prices[start_date + 4] > 0:
        phase_price.append(this_week_prices[start_date + 4])
    else:
        max_price, min_price = calculate_max_min_price(increasing_rates_1)
        phase_price.append(print_range(min_price, max_price))
    return phase_price

# 第三个下跌区间
def process_decreasing_phase_3(start_date, end_date):
    return calculate_continuous_phases_prices(start_date, end_date,[0.4, 0.9])

# 三期型算法
def calculate_large_spike():
    result = []
    max_decreasing_period_1_phases = 7
    increasing_period_1_phases = 3
    decreasing_period_2_phases = 2
    decreased_days, decreasing_phase_1_end = calculate_increased_or_decreased_days(increasing_or_decreasing.decreasing)
    
    if decreasing_phase_1_end:
        decreasing_phase_1 = process_decreasing_phase_1(0, decreased_days)
        incressing_phase_1_and_decreasing_phase_2 = process_increasing_phase_1_and_decreasing_phase_2(decreased_days)
        decreasing_phase_3 = process_decreasing_phase_3(
            decreased_days + increasing_period_1_phases + decreasing_period_2_phases, total_phases
        )
        result.append(decreasing_phase_1 + incressing_phase_1_and_decreasing_phase_2 + decreasing_phase_3)
    else:
        for i in range(decreased_days, max_decreasing_period_1_phases + 1):
            decreasing_phase_1 = process_decreasing_phase_1(0, i)
            incressing_phase_1_and_decreasing_phase_2 = process_increasing_phase_1_and_decreasing_phase_2(i)
            decreasing_phase_3 = process_decreasing_phase_3(i + increasing_period_1_phases + decreasing_period_2_phases, total_phases)
            result.append(decreasing_phase_1 + incressing_phase_1_and_decreasing_phase_2 + decreasing_phase_3)
    return result

In [26]:
'''
四期型处理
分为3个区间：下跌区间1 - 上涨区间1 - 下跌区间2
时间上分别是 0 到 7 个半天 - 5 个半天 - 剩余半天数量
'''
# 第一个下跌区间
def process_decreasing_phase_1(start_date, end_date):
    return calculate_continuous_phases_prices(start_date, end_date,[basic_rate], [0.03, 0.05])
    
# 第一个上涨区间
def process_incressing_phase_1(start_date):
    increasing_rates_1 = [0.9, 1.4]
    increasing_rates_2 = [1.4, 2.0]
    phase_price = []

    # 上涨第一天
    if this_week_prices[start_date] > 0:
        phase_price.append(this_week_prices[start_date])
    else:
        max_price, min_price = calculate_max_min_price(increasing_rates_1)
        phase_price.append(print_range(min_price, max_price))

    # 上涨第二天
    if this_week_prices[start_date + 1] > 0:
        phase_price.append(this_week_prices[start_date + 1])
    else:
        max_price, min_price = calculate_max_min_price(increasing_rates_1)
        phase_price.append(print_range(min_price, max_price))

    # 上涨第三天
    if this_week_prices[start_date + 2] > 0:
        phase_price.append(this_week_prices[start_date + 2])
    else:
        max_price, min_price = calculate_max_min_price(increasing_rates_2)
        phase_price.append(print_range(min_price - 1, max_price - 1))    

    # 上涨第四天
    if this_week_prices[start_date + 3] > 0:
        phase_price.append(this_week_prices[start_date + 3])
    else:
        max_price, min_price = calculate_max_min_price(increasing_rates_2)
        phase_price.append(print_range(min_price, max_price))   

    # 上涨第五天
    if this_week_prices[start_date + 4] > 0:
        phase_price.append(this_week_prices[start_date + 4])
    else:
        max_price, min_price = calculate_max_min_price(increasing_rates_2)
        phase_price.append(print_range(min_price - 1, max_price - 1))    
    return phase_price

# 第二个下跌区间
def process_decreasing_phase_2(start_date, end_date):
    return calculate_continuous_phases_prices(start_date, end_date, [0.4, 0.9], [0.03, 0.05], start_date)

# 四期型算法
def calculate_small_spike():
    result = []
    max_decreasing_period_1_phases = 7
    increasing_period_1_phases = 5
    decreased_days, decreasing_phase_1_end = calculate_increased_or_decreased_days(increasing_or_decreasing.decreasing)
    
    if decreasing_phase_1_end:
        decreasing_phase_1 = process_decreasing_phase_1(0, decreased_days)
        incressing_phase_1 = process_incressing_phase_1(decreased_days)
        decreasing_phase_2 = process_decreasing_phase_2(decreased_days + increasing_period_1_phases, total_phases)
        result.append(decreasing_phase_1 + incressing_phase_1 + decreasing_phase_2)
    else:
        for i in range(decreased_days, max_decreasing_period_1_phases + 1):
            decreasing_phase_1 = process_decreasing_phase_1(0, i)
            incressing_phase_1 = process_incressing_phase_1(i)
            decreasing_phase_2 = process_decreasing_phase_2(i + increasing_period_1_phases, total_phases)
            result.append(decreasing_phase_1 + incressing_phase_1 + decreasing_phase_2)
    return result

In [27]:
# 判断走势模型
pricing_patterns_this_week = set()
if basic_rate > 0.9:
    # 四期型 or 波动型（概率较高）
    pricing_patterns_this_week.add(pricing_patterns[0])
    pricing_patterns_this_week.add(pricing_patterns[3])
    if this_week_prices[1] != 0 and this_week_prices[1] / last_sundy_price < 0.8:
        pricing_patterns_this_week.clear()
        pricing_patterns_this_week.add(pricing_patterns[0])
    elif this_week_prices[1] != 0 and this_week_prices[1] != 0 and this_week_prices[2] / last_sundy_price > 1.4:
        pricing_patterns_this_week.clear()
        pricing_patterns_this_week.add(pricing_patterns[0])
    elif this_week_prices[1] != 0 and this_week_prices[1] != 0:
        pricing_patterns_this_week.clear()
        pricing_patterns_this_week.add(pricing_patterns[3])
elif basic_rate > 0.85 and basic_rate < 0.9:
    # 递减型 or 三期型 or 四期型
    pricing_patterns_this_week.add(pricing_patterns[1])
    pricing_patterns_this_week.add(pricing_patterns[2])
    pricing_patterns_this_week.add(pricing_patterns[3])
    for i,j in enumerate(range(1, 8)):
        # 直到周四下午还一直下跌：递减型
        if this_week_prices[i] != 0 and this_week_prices[j] != 0 and this_week_prices[i] > this_week_prices[j] and j == 7:
            pricing_patterns_this_week.clear()
            pricing_patterns_this_week.add(pricing_patterns[1])
        if this_week_prices[i] < this_week_prices[j]:
            r = this_week_prices[j] / this_week_prices[i]
            # 开始涨的第二期涨幅超过1.4：三期型，否则： 四期型
            if r > 1.4:
                pricing_patterns_this_week.clear()
                pricing_patterns_this_week.add(pricing_patterns[2])
                break;
            else:
                pricing_patterns_this_week.clear()
                pricing_patterns_this_week.add(pricing_patterns[3])
                break;
elif basic_rate > 0.8 and basic_rate < 0.85:
    # 四期型
    pricing_patterns_this_week.add(pricing_patterns[3])
elif basic_rate > 0.6 and basic_rate < 0.8:
    # 四期型 or 波动型（概率较高）
    pricing_patterns_this_week.add(pricing_patterns[0])
    pricing_patterns_this_week.add(pricing_patterns[3])
    if this_week_prices[1] != 0 and this_week_prices[1] / last_sundy_price < 0.8:
        pricing_patterns_this_week.clear()
        pricing_patterns_this_week.add(pricing_patterns[0])
    elif this_week_prices[1] != 0 and this_week_prices[1] != 0 and this_week_prices[2] / last_sundy_price > 1.4:
        pricing_patterns_this_week.clear()
        pricing_patterns_this_week.add(pricing_patterns[0])
    elif this_week_prices[2] !=0:
        pricing_patterns_this_week.clear()
        pricing_patterns_this_week.add(pricing_patterns[3])
elif basic_rate < 0.6:
    # 四期型
    pricing_patterns_this_week.add(pricing_patterns[3])

pricing_patterns_last_week.append(pricing_patterns_this_week)
pricing_patterns_this_week

{'三期型'}

In [28]:
# 分模型处理
results = []
results_idx = []
for item in pricing_patterns_this_week:    
    if item == pricing_patterns[0]:
        prediction_result = calculate_random()
        results_idx = results_idx + [str(last_sunday_price) + ' : ' + item] * len(prediction_result)
        results = results + prediction_result
    elif item == pricing_patterns[1]:
        prediction_result = calculate_decreasing()
        results_idx = results_idx + [str(last_sunday_price) + ' : ' + item] * len(prediction_result)
        results = results + prediction_result
    elif item == pricing_patterns[2]:
        prediction_result = calculate_large_spike()
        results_idx = results_idx + [str(last_sunday_price) + ' : ' + item] * len(prediction_result)
        results = results + prediction_result
    elif item == pricing_patterns[3]:
        prediction_result = calculate_small_spike()
        results_idx = results_idx + [str(last_sunday_price) + ' : ' + item] * len(prediction_result)
        results = results + prediction_result

In [29]:
df = pandas.DataFrame(
    data = results,
    index = results_idx,
    columns = [
        '周一上午', '周一下午','周二上午', '周二下午',
        '周三上午', '周三下午','周四上午', '周四下午',
        '周五上午', '周五下午','周六上午', '周六下午'
    ]
)

In [30]:
df

1


,周一上午,周一下午,周二上午,周二下午,周三上午,周三下午,周四上午,周四下午,周五上午,周五下午,周六上午,周六下午
101 : 三期型,88,83,80,127,154,202-606,142-202,91-142,41-91,41-91,41-91,41-91
